In [19]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 2 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 2080 SUPER


#installing dependencies

In [20]:
!pip install optuna==2.3.0
!pip install transformers==4.2.1
!pip install farasapy
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert

  Using cached optuna-2.3.0-py3-none-any.whl
  Attempting uninstall: optuna
    Found existing installation: optuna 2.10.0
    Uninstalling optuna-2.10.0:


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\optuna-2.10.0.dist-info\\entry_points.txt'
Consider using the `--user` option or check the permissions.



  Using cached transformers-4.2.1-py3-none-any.whl (1.8 MB)
  Using cached tokenizers-0.9.4-cp38-cp38-win_amd64.whl (1.9 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\~okenizers\\tokenizers.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



fatal: destination path 'arabert' already exists and is not an empty directory.


In [21]:
!pip install openpyxl
!pip install xlrd

#Creating training datasets

In [22]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
all_datasets= []

In [43]:
class Dataset:
    def __init__(
        self,
        name,
        train,
        test,
        label_list,
    ):
        self.name = name
        self.train = train
        self.test = test
        self.label_list = label_list

##HARD - Balanced

In [44]:
df_HARD = pd.read_excel('ml2.xlsx', engine='openpyxl', header=None)
df_HARD = df_HARD.rename(columns={0: 'tags', 1: 'post'})
print(df_HARD["tags"].value_counts())
# code rating as +ve if > 3, -ve if less, no 3s in dataset

train_HARD, test_HARD = train_test_split(df_HARD, test_size=0.2, random_state=42)
label_list_HARD = list(range(1, 14))

data_Hard = Dataset("ml2", train_HARD, test_HARD, label_list_HARD)
all_datasets.append(data_Hard)

13    1025
12     998
11     755
10     646
9      574
8      482
7      477
3      440
5      425
6      414
4      410
1      304
2      292
Name: tags, dtype: int64


In [45]:
for x in all_datasets:
  print(x.name) 

ML1
ml2


#Trainer

In [46]:
!pip install transformers -U

In [47]:
from arabert.preprocess import ArabertPreprocessor
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample
import logging
import torch
import optuna 

In [48]:
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

In [49]:
for x in all_datasets:
  print(x.name)

ML1
ml2


You can choose which model, and dataset from here along with the max sentence length

In [50]:
dataset_name = 'ml2'
model_name = 'aubmindlab/bert-base-arabertv2'
task_name = 'classification'
max_len = 256

In [51]:
for d in all_datasets:
  if d.name==dataset_name:
    selected_dataset = d
    print('Dataset found')
    break

Dataset found


In [52]:
arabert_prep = ArabertPreprocessor("bert-base-arabertv2")

selected_dataset.train['post'] = selected_dataset.train['post'].apply(lambda x: arabert_prep.preprocess(x))
selected_dataset.test['post'] = selected_dataset.test['post'].apply(lambda x: arabert_prep.preprocess(x))  

[2022-06-28 18:48:31,275 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
<ipython-input-52-db95ccf0e1e4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_dataset.train['post'] = selected_dataset.train['post'].apply(lambda x: arabert_prep.preprocess(x))
<ipython-input-52-db95ccf0e1e4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_dataset.test['post'] = selected_dataset.test['post'].apply(la

In [53]:
class BERTDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=self.label_map[self.target[item]])

In [54]:
label_map = { v:index for index, v in enumerate(selected_dataset.label_list) }
print(label_map)
train_dataset = BERTDataset(selected_dataset.train['post'].to_list(),selected_dataset.train['tags'].to_list(),model_name,max_len,label_map)
test_dataset = BERTDataset(selected_dataset.test['post'].to_list(),selected_dataset.test['tags'].to_list(),model_name,max_len,label_map)

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12}


loading configuration file https://huggingface.co/aubmindlab/bert-base-arabertv2/resolve/main/config.json from cache at C:\Users\bwfch/.cache\huggingface\transformers\fcf7a61cc072840ad32e1a2e8eb230a79b33db68e3f965f8014a52915cab999f.2f0d0092105af7b8b42b899ffb7f801dc48e93516d509483f6cfbd86155d49ea
Model config BertConfig {
  "_name_or_path": "aubmindlab/bert-base-arabertv2",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 64000
}

loading file https://huggingface.co/aubmindlab/bert-base-arabert

In [35]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [36]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  #print(classification_report(p.label_ids,preds))
  #print(confusion_matrix(p.label_ids,preds))

  macro_f1_per_label = f1_score(p.label_ids, preds,average='macro', labels=list(range(1, 14)))
  macro_f1 = f1_score(p.label_ids,preds, average='macro')
  macro_precision = precision_score(p.label_ids, preds,average='macro')
  macro_recall = recall_score(p.label_ids, preds, average='macro')
  acc = accuracy_score(p.label_ids, preds)
  return {
      'macro_f1' : macro_f1,
      'macro_f1_per_label' : macro_f1_per_label,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
  }

# Regular Training

This paert allows you to do a regular training with no hyper parameter optimization

In [56]:
training_args = TrainingArguments(
    output_dir= "./train",
    do_eval = True,
    do_train = True,
    evaluation_strategy = 'epoch',
    adam_epsilon = 1e-8,
    learning_rate = 5e-5,
    fp16 = False,
    per_device_train_batch_size =8,
    per_device_eval_batch_size = 8,
    gradient_accumulation_steps = 2,
    num_train_epochs= 8,
    warmup_ratio =0,
    save_strategy = 'epoch',
    seed = 42,
    lr_scheduler_type = 'cosine'
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [57]:
training_args.n_gpu

2

In [58]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)


loading configuration file https://huggingface.co/aubmindlab/bert-base-arabertv2/resolve/main/config.json from cache at C:\Users\bwfch/.cache\huggingface\transformers\fcf7a61cc072840ad32e1a2e8eb230a79b33db68e3f965f8014a52915cab999f.2f0d0092105af7b8b42b899ffb7f801dc48e93516d509483f6cfbd86155d49ea
Model config BertConfig {
  "_name_or_path": "aubmindlab/bert-base-arabertv2",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11":

In [59]:
trainer.train()

***** Running training *****
  Num examples = 5793
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 1448
C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
C:\ProgramData\Anaconda3\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')


Epoch,Training Loss,Validation Loss,Macro F1,Macro F1 Per Label,Macro Precision,Macro Recall,Accuracy
0,No log,0.316474,0.892387,0.821484,0.898812,0.889899,0.912353
1,No log,0.290664,0.904875,0.833257,0.909789,0.904148,0.922015
2,0.392200,0.320950,0.897309,0.827875,0.903060,0.893302,0.913734
3,0.392200,0.348056,0.895013,0.823218,0.903300,0.891022,0.913043
4,0.392200,0.340307,0.898512,0.826191,0.903346,0.896184,0.915114
5,0.123200,0.360921,0.891652,0.819944,0.893854,0.891086,0.909593
6,0.123200,0.374425,0.894347,0.822639,0.895864,0.893976,0.911663
7,0.123200,0.379476,0.895859,0.824151,0.897727,0.894985,0.913043


***** Running Evaluation *****
  Num examples = 1449
  Batch size = 16
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1492: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
Saving model checkpoint to ./train\checkpoint-181
Configuration saved in ./train\checkpoint-181\config.json
Model weights saved in ./train\checkpoint-181\pytorch_model.bin
C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
C:\ProgramData\Anaconda3\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
***** Running Evaluation *****


TrainOutput(global_step=1448, training_loss=0.19394024432693396, metrics={'train_runtime': 1349.1834, 'train_samples_per_second': 34.35, 'train_steps_per_second': 1.073, 'total_flos': 6097279954985472.0, 'train_loss': 0.19394024432693396, 'epoch': 8.0})

In [60]:
trainer.save_model("ml2")

Saving model checkpoint to ml2
Configuration saved in ml2\config.json
Model weights saved in ml2\pytorch_model.bin


In [61]:
train_dataset.tokenizer.save_pretrained("ml2")

tokenizer config file saved in ml2\tokenizer_config.json
Special tokens file saved in ml2\special_tokens_map.json


('ml2\\tokenizer_config.json',
 'ml2\\special_tokens_map.json',
 'ml2\\vocab.txt',
 'ml2\\added_tokens.json',
 'ml2\\tokenizer.json')